<a href="https://colab.research.google.com/github/Alice-zou/Natural-Disaster-Chatbot/blob/main/Dataset_QA_Generation_Alice_jsonl_multipdf_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_openai install langchain langchain-community pypdf PyPDF2 sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 741.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
import openai
import json
import time
import os

In [ ]:
from google.colab import drive
# Mounts Google Drive to the Colab environment
drive.mount('/content/drive')

# Define the path to the folder in Google Drive where the JSON file is saved
drive_folder_path = '/content/drive/My Drive/NeurIPS Outreach/Dataset/Alice Zou - Subsidence'

# Generate a new folder if the destination does not already exists
if not os.path.exists(drive_folder_path):
    os.makedirs(drive_folder_path)
os.chdir(drive_folder_path)
!pwd

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import langchain
from langchain_openai import OpenAI
import re
from typing import List, Tuple
import os

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
from IPython.display import display
import pandas as pd
from PyPDF2 import PdfMerger, PdfWriter

In [ ]:
# IMPORTANT! Replace with your own API key.
os.environ["OPENAI_API_KEY"] = 'Your Open AI Key'
# Add your file path here. Don't forget to specify a good file name.
file_paths = ["/content/drive/MyDrive/NeurIPS Outreach/Dataset/Alice Zou - Subsidence/downloaded research papers/5.pdf"]
#filename: The name of the output JSON file.
filename="qa_output1.jsonl"

merger = PdfMerger()
for file_path in file_paths:
  merger.append(file_path)
merger.write("document-output.pdf")

# Load documents
loader = PyPDFLoader("document-output.pdf")
documents = loader.load()

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# Create index
embeddings = HuggingFaceEmbeddings()
index = VectorstoreIndexCreator(embedding=embeddings).from_loaders([loader])

# Initialize LLM (replace with your preferred LLM)
llm = OpenAI(model="gpt-3.5-turbo-instruct")

# Load QA chain with source
chain = load_qa_with_sources_chain(llm, chain_type="stuff")

# Generate QA pairs
qa_pairs = []



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/indexes/vectorstore.py:129: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [ ]:
for i in range(len(texts)):

  # Generate question from the text chunk
  question_prompt = f"Generate a question based on the following text:\n\n{texts[i].page_content}. Do not provide sources."
  question_result = llm(question_prompt)
  question = question_result.strip()  # Extract the generated question

  answer = chain({"question": question + " Do not include your sources in your answer.", "input_documents": [texts[i]]})
  qa_pairs.append({"messages": [{"role": "system", "content": "Scien is a helpful chatbot that tries to provide accurate answers to questions regarding natual disasters."}, {"role": "user", "content": question}, {"role": "assistant", "content": answer['output_text']}]})

# Save QA pairs to JSON file
with open(filename, 'w') as f:
    for item in qa_pairs:
        json.dump(item, f)
        f.write('\n')

# Display JSON in a dataframe.
print(f"JSON file '{filename}' created successfully.")

JSON file 'qa_output1.jsonl' created successfully.
